# Catboost Hyperparameter Sweep 20210920
Largely after https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu

# Setup

In [1]:
# two manual flags (ex-config)
colab = False
gpu_available = False

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random
import multiprocessing
import pickle

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"sweep_catboost_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if colab:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
    # !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # !pip install --upgrade xgboost

    # upgrade sklearn
    !pip install --upgrade scikit-learn

    # !pip install category_encoders
    !pip install catboost

    # lighgbm gpu compatible
    # !git clone --recursive https://github.com/Microsoft/LightGBM
    # ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
    
    # # this part is from https://github.com/rapidsai/gputreeshap/issues/24
    # !pip install cmake --upgrade
    # # !pip install sklearn --upgrade
    # !git clone --recursive https://github.com/dmlc/xgboost
    # %cd /content/xgboost
    # !mkdir build
    # %cd build
    # !cmake .. -DUSE_CUDA=ON
    # !make -j4
    # %cd /content/xgboost/python-package
    # !python setup.py install --use-cuda --use-nccl
    # !/opt/bin/nvidia-smi
    # !pip install shap
    

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from optuna.integration.wandb import WeightsAndBiasesCallback
# from wandb.xgboost import wandb_callback
# from wandb.lightgbm import wandb_callback
# from sklearn.impute import KNNImputer, StandardImputer
# import timm

import seaborn as sns

from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


from optuna.samplers import TPESampler
import optuna
import catboost
from sklearn.utils import resample
import sklearn.metrics

Now, datapath setup

In [6]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

In [7]:
if colab:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/sep2021/')
    
else:
    # if on local machine
    datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')    
    


In [8]:

# n_trials = int(1000)
SEED = 42

In [9]:
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(SEED)

## Ex-Model Config

In [10]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
# in the sweep version, this includes both ex-model parameters and defaults for model parameters
exmodel_config = {
    # model config
    "library": 'catboost',
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
    "scaler": "sklearn.preprocessing.StandardScaler()", # TODO: experiment with others (but imputation may be slow)
    "scale_b4_impute": False,
    "imputer": "sklearn.impute.SimpleImputer(strategy='median', add_indicator=True)",
    "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': SEED,
    'optuna': True,
    'optuna_trials': 100,
#     'subsample': 1,
#     'cross_val_strategy': None, # None for holdout, or the relevant sklearn class
#     'kfolds': 1, # if 1, that means just doing holdout
#     'test_size': 0.2,
    # these are XGBoost default (my choice) params 
#     "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
#     "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
#     "n_estimators": 200, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "n_jobs": -1,
#     "verbosity": 1,
#     "subsample": 1,
#     'features_created': False,
#     'feature_creator': None,
}

wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202109_Kaggle_tabular_playground',
    'tags': ['sweep'],
    'notes': "Sweep for CatBoost using Optuna",
    'config': exmodel_config,
}

## Data Setup

**TODO** Write some conditional logic here to automate it -- possibly as part of a sklearn.*pipeline

In [11]:
X_source = 'X_NaNcounts_SummaryStats_imputed-Median-wIndicators-StandardScaled.feather'
X_train = pd.read_feather(datapath/X_source) 
y_train = load(datapath/'y.joblib')    
# X.index.name = 'id'
# y.index.name = 'id'
X = np.array(X_train)[:50000]
y = np.array(y_train)[:50000]

del X_train, y_train

In [12]:
exmodel_config['feature_count'] = X.shape[1]
exmodel_config['feature_generator'] = "Summary statistics"
exmodel_config['instance_count'] = X.shape[0]
exmodel_config['X_source'] = X_source

# Experiment setup

In [13]:
# wandb_kwargs = {
#     # wandb config:
#     'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
#     'project': '202109_Kaggle_tabular_playground',
#     'tags': ['sweep'],
#     'notes': "Sweep for CatBoost using Optuna",
#     'config': exmodel_config,
# }

A basis for the experimental ranges, taken from [here](https://www.kaggle.com/vishwas21/tps-sep-21-3-level-custom-stacking):

```python
catb_params = {
    'iterations': 15585, 
    'objective': 'CrossEntropy', 
    'bootstrap_type': 'Bernoulli', 
    'od_wait': 1144, 
    'learning_rate': 0.023575206684596582, 
    'reg_lambda': 36.30433203563295, 
    'random_strength': 43.75597655616195, 
    'depth': 7, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 1, 
    'subsample': 0.8227911142845009,
    'task_type' : 'GPU',
    'devices' : '0',
    'verbose' : 0
}
```

In [14]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial):
    # split the (original Kaggle training) data into partitions
    # if study.best_trial:
    #     print("Dumping best params, which are:")
    #     print(str(study.best_trial.params))
    #     dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20210920.joblib')
       
    # else:
    #     print("No best study yet")
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=int(SEED), shuffle=True)
    # create wrappers for the training and validation partitions
    train_pool = catboost.Pool(X_train, y_train)
    valid_pool = catboost.Pool(X_valid, y_valid)
    
    # experimental parameters
    params = {
        'iterations' : trial.suggest_int('iterations', 2000, 30000),                         
        'depth' : trial.suggest_int('depth', 3, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.5),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100), 
#         'objective': trial.suggest_categorical('objective', ['Logloss', 'CrossEntropy']),
#         'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['MVC', 'Bernoulli']),#, 'Poisson']),
        'od_wait': trial.suggest_int('od_wait', 20, 2000),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 2, 70), # aka l2_leaf_reg
        'border_count': trial.suggest_int('border_count', 50, 275),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20), # aka min_data_in_leaf
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 5),
        # 'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1),
#         'learning_rate' :trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        # 'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#         'max_leaves': trial.suggest_int('max_leaves', 32, 128)
    }
    device = "GPU" if gpu_available else "CPU"
    # instantiate the model, with some parameters locked in, and experimnental ones passed via splat 
    model = CatBoostClassifier(
        # loss_function="Logloss",
        eval_metric="AUC",
        task_type=device,
        objective='Logloss',
#         l2_leaf_reg=50,
        random_seed=SEED,
        verbose=False,
#         border_count=64,
        **params
    )        
#     wandb.init(wandb_kwargs)
    # fit the model on the training set -- note the use of the catboost.Pool instance
    model.fit(train_pool)
    # generate predictions
    preds = model.predict_proba(valid_pool)[:,1]
    # rounds to the nearest integer, and the nearest even in case of _.5s
    # pred_labels = np.rint(preds)
    # calls sklearn.utils.resample... not entirely sure why
    # y_pred_boot = resample(pred_labels, n_samples = len(train_y))
    # Evaluation
    # ROC_AUC_Score = roc_auc_score(train_y, y_pred_boot)
    valid_auc = roc_auc_score(y_valid, preds)
    
    print('ROC AUC Score of CatBoost =', valid_auc)
    wandb.log({'valid_AUC': valid_auc,
            #    'catboost_params': str(model.get_all_params())
              })
#     wandb.finish()
    return valid_auc

In [15]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

<ipython-input-15-69ea9289a2cf>:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)


In [16]:
study = optuna.create_study(direction = "maximize", sampler = TPESampler(seed=int(SEED)), study_name='CatBoost_20210924')
# setting n_jobs=1 bc wandb apparently requires this


[I 2021-09-24 22:31:13,738] A new study created in memory with name: CatBoost_20210924


In [17]:
for x in range(100):
    study.optimize(objective, n_trials = 1, callbacks = [wandbc]) #n_jobs = multiprocessing.cpu_count())
    dump(study, filename=datapath/f'optuna_catboost_study_thru_trial{x}_20210924.joblib')

[I 2021-09-24 22:39:23,592] Trial 0 finished with value: 0.796746455020556 and parameters: {'iterations': 12487, 'depth': 10, 'learning_rate': 0.09454306819536169, 'random_strength': 60, 'od_wait': 329, 'reg_lambda': 12.60762738286178, 'border_count': 63, 'min_child_samples': 18, 'leaf_estimation_iterations': 4, 'subsample': 0.8540362888980227}. Best is trial 0 with value: 0.796746455020556.
wandb: WARNING Step must only increase in log calls.  Step 0 < 1; dropping {'iterations': 12487, 'depth': 10, 'learning_rate': 0.09454306819536169, 'random_strength': 60, 'od_wait': 329, 'reg_lambda': 12.60762738286178, 'border_count': 63, 'min_child_samples': 18, 'leaf_estimation_iterations': 4, 'subsample': 0.8540362888980227, 'value': 0.796746455020556}.


ROC AUC Score of CatBoost = 0.796746455020556


[I 2021-09-24 22:41:28,077] Trial 1 finished with value: 0.7931660095091524 and parameters: {'iterations': 2576, 'depth': 10, 'learning_rate': 0.1764971584817571, 'random_strength': 21, 'od_wait': 380, 'reg_lambda': 14.471506670033499, 'border_count': 118, 'min_child_samples': 11, 'leaf_estimation_iterations': 3, 'subsample': 0.645614570099021}. Best is trial 0 with value: 0.796746455020556.
wandb: WARNING Step must only increase in log calls.  Step 1 < 2; dropping {'iterations': 2576, 'depth': 10, 'learning_rate': 0.1764971584817571, 'random_strength': 21, 'od_wait': 380, 'reg_lambda': 14.471506670033499, 'border_count': 118, 'min_child_samples': 11, 'leaf_estimation_iterations': 3, 'subsample': 0.645614570099021, 'value': 0.7931660095091524}.


ROC AUC Score of CatBoost = 0.7931660095091524


[I 2021-09-24 22:42:50,426] Trial 2 finished with value: 0.8011908546666167 and parameters: {'iterations': 19132, 'depth': 4, 'learning_rate': 0.00614454378558747, 'random_strength': 37, 'od_wait': 923, 'reg_lambda': 55.391965374724926, 'border_count': 95, 'min_child_samples': 11, 'leaf_estimation_iterations': 3, 'subsample': 0.5232252063599989}. Best is trial 2 with value: 0.8011908546666167.
wandb: WARNING Step must only increase in log calls.  Step 2 < 3; dropping {'iterations': 19132, 'depth': 4, 'learning_rate': 0.00614454378558747, 'random_strength': 37, 'od_wait': 923, 'reg_lambda': 55.391965374724926, 'border_count': 95, 'min_child_samples': 11, 'leaf_estimation_iterations': 3, 'subsample': 0.5232252063599989, 'value': 0.8011908546666167}.


...

ROC AUC Score of CatBoost = 0.8013854092797392


[I 2021-09-25 01:42:19,754] Trial 98 finished with value: 0.7951127164148244 and parameters: {'iterations': 5052, 'depth': 7, 'learning_rate': 0.06381613345558103, 'random_strength': 7, 'od_wait': 1992, 'reg_lambda': 38.10008746358754, 'border_count': 252, 'min_child_samples': 5, 'leaf_estimation_iterations': 2, 'subsample': 0.5378564167600625}. Best is trial 91 with value: 0.8029241260077378.
wandb: WARNING Step must only increase in log calls.  Step 98 < 99; dropping {'iterations': 5052, 'depth': 7, 'learning_rate': 0.06381613345558103, 'random_strength': 7, 'od_wait': 1992, 'reg_lambda': 38.10008746358754, 'border_count': 252, 'min_child_samples': 5, 'leaf_estimation_iterations': 2, 'subsample': 0.5378564167600625, 'value': 0.7951127164148244}.


ROC AUC Score of CatBoost = 0.7951127164148244


[I 2021-09-25 01:42:52,439] Trial 99 finished with value: 0.800192676803323 and parameters: {'iterations': 2732, 'depth': 6, 'learning_rate': 0.03221334877855445, 'random_strength': 4, 'od_wait': 1709, 'reg_lambda': 32.70388391136099, 'border_count': 271, 'min_child_samples': 6, 'leaf_estimation_iterations': 2, 'subsample': 0.6484246829473108}. Best is trial 91 with value: 0.8029241260077378.
wandb: WARNING Step must only increase in log calls.  Step 99 < 100; dropping {'iterations': 2732, 'depth': 6, 'learning_rate': 0.03221334877855445, 'random_strength': 4, 'od_wait': 1709, 'reg_lambda': 32.70388391136099, 'border_count': 271, 'min_child_samples': 6, 'leaf_estimation_iterations': 2, 'subsample': 0.6484246829473108, 'value': 0.800192676803323}.


ROC AUC Score of CatBoost = 0.800192676803323


In [18]:
dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20210920.joblib')
# pickle.dump(study.best_trial.params, open('CatBoost_Hyperparameter.pickle', 'wb'))
# print('CatBoost Hyperparameter:', study.best_trial.params)

['/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/optuna_catboost_best_20210920.joblib']

In [19]:
wandb.log({'catboost_params': study.best_trial.params})
wandb.finish()

valid_AUC,0.80019


valid_AUC,▄▇▁▆▅▇▄▅▇▇▆▅▇▇▅▇▂▅▇▄▄▇▇▇▅▇▇▇▇█▇▇▇██▅█▇▇▆


In [20]:
study.best_trial.params

{'iterations': 2028,
 'depth': 5,
 'learning_rate': 0.01894945292658026,
 'random_strength': 4,
 'od_wait': 1895,
 'reg_lambda': 36.31771727215521,
 'border_count': 263,
 'min_child_samples': 5,
 'leaf_estimation_iterations': 2,
 'subsample': 0.5882959573476197}